加载必要的例库

In [1]:
import paddle.fluid as fluid
import numpy as np
import paddle

定义模型结构，与之前训练是用的模型相同。[参考训练教程](hhttps://aistudio.baidu.com/aistudio/projectdetail/628079)

In [2]:
import paddle.fluid as fluid
from paddle.fluid.dygraph.nn import Conv2D, Pool2D, Linear

# 定义 CNN 网络结构
class CNN(fluid.dygraph.Layer):
    def __init__(self, num_classes=1):
        super(CNN, self).__init__()

        # 创建卷积和池化层块，每个卷积层使用Sigmoid激活函数，后面跟着一个2x2的池化
        self.conv1 = Conv2D(num_channels=1, num_filters=32, filter_size=3, act='sigmoid')
        self.pool1 = Pool2D(pool_size=2, pool_stride=1, pool_type='max')
        self.conv2 = Conv2D(num_channels=32, num_filters=64, filter_size=3, act='sigmoid')
        self.pool2 = Pool2D(pool_size=2, pool_stride=1, pool_type='max')
        self.fc1 = Linear(input_dim=30976, output_dim=num_classes)

    # 网络的前向计算过程
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = fluid.layers.reshape(x, [x.shape[0], -1])
        x = self.fc1(x)
        return x



定义测试函数

In [3]:
def test(model):
    model.eval()
    valid_loader = paddle.batch(paddle.dataset.mnist.test(), batch_size=10)
    accuracies = []
    losses = []
    for batch_id, data in enumerate(valid_loader()):
        # 调整输入数据形状和类型
        x_data = np.array([item[0] for item in data], dtype='float32').reshape(-1, 1, 28, 28)
        y_data = np.array([item[1] for item in data], dtype='int64').reshape(-1, 1)
        # 将numpy.ndarray转化成Tensor
        img = fluid.dygraph.to_variable(x_data)
        label = fluid.dygraph.to_variable(y_data)
        # 计算模型输出
        logits = model(img)
        pred = fluid.layers.softmax(logits)
        # 计算损失函数
        loss = fluid.layers.softmax_with_cross_entropy(logits, label)
        acc = fluid.layers.accuracy(pred, label)
        accuracies.append(acc.numpy())
        losses.append(loss.numpy())
    print("[validation] accuracy/loss: {}/{}".format(np.mean(accuracies), np.mean(losses)))

初始化模型
>model = CNN(num_classes=10)

加载参数
> para_state_dict, opti_state_dict = fluid.load_dygraph( "mnist")
> model.set_dict(para_state_dict)

测试
> test(model)

In [4]:
if __name__ == '__main__':
    # 创建模型
    with fluid.dygraph.guard():
        model = CNN(num_classes=10)
        para_state_dict, opti_state_dict = fluid.load_dygraph( "mnist")
        model.set_dict(para_state_dict)
        #启动训练过程
        test(model)

[validation] accuracy/loss: 0.9817999005317688/0.05483948811888695
